[2024-10-04 Fiddler](https://thefiddler.substack.com/p/how-many-dice-can-you-roll-the-same)
====================

It's not specified anywhere, so I'll assume that the dice are 6-sided.

The question seems ambiguous.  I can see three interpretations.

The obviously wrong interpretation has an easy answer.  The total number of dice set aside per player
per game on average will be 3, or, for the extra credit, 10 or $N$.

The second interpretation is the number of dice set aside from the first roll on average.
This is one roll per player per game.

The third interpretation is the average number of dice set aside per roll from each roll in the game,
including the first roll.

The wording seems to seems to imply the second interpretation.  Also, the first and third interpretations
can only be calculated for the winning player if the losing players stop rolling once they lose.  (For the
second interpretation, a very slow losing player might lose before setting aside the dice from the first
roll, and thus not contribute to the average.)

Simulation
----------
To get a sense of what the answers should be, run some [simulations](20241004.go), calculating both
the average number of dice set aside from the first roll of a game and the average number of dice
set aside per roll from every roll.

    $ go run 20241004.go
    3: 1.472164 0.359562
    10: 3.445436 0.651536
    25: 6.899469 1.208623
    50: 12.143944 2.026984
    100: 21.998333 3.496237

Fiddler
-------

The probability of rolling 3 of the same number is $1/36$.  The probability of rolling 3 different
numbers is $20/36$, so the probability of rolling 2 of one number and 1 of a different number is
$15/36$.  That means the average number of dice set aside from the first roll is

In [1]:
x = 3*1/36 + 2*15/36 + 1*20/36
(x,numerical_approx(x))

(53/36, 1.47222222222222)

which agrees with the result of the simulations.

When there is 1 die left, the average number of rolls is $6$.

When there are 2 dice left, the probability of setting aside 2 dice in a roll is $1/36$.  The
probability of setting aside 1 die is $10/36$.  The probability of setting aside none is $25/36$,
so the average number of rolls is

In [2]:
x = var('x')
solve(x == 1 + 0*1/36 + 6*10/36 + x*25/36, x)

[x == (96/11)]

The overall average number of rolls is then

In [3]:
1*1/36 + (1 + 6)*15/36 + (1 + 96/11)*20/36

551/66

So the overall average number of dice set aside per roll is

In [4]:
x = 3/(551/66)
(x,numerical_approx(x))

(198/551, 0.359346642468240)

which agrees with the result of the simulations.

Extra credit
------------

Let $P(N,s,n,k)$ be the probability of setting aside $n$ dice out of $N$ dice with $s$ sides
when there are $k$ groups of $n$ dice with the same face.

The average number of dice set aside from the first roll is
$\sum_{n=1}^{N} \sum_{k=1}^{6} nP(N,6,n,k)$.

Let $R(n)$ be the average number of rolls to set aside $n$ dice after the target is set.

The average number of dice set aside per roll is
$N/\sum_{n=1}^{N} \sum_{k=1}^{6} (1+R(N-n))P(N,6,n,k)$.

#### Consider $N=10$
To get a better sense of how to approach the general case, consider $N=10$.

The probabilities of setting aside 6, 7, 8, 9, and 10 from the first roll are

In [5]:
p10 = binomial(6,1)*(1/6)^10*binomial(10,10)
p9 = binomial(6,1)*(1/6)^9*binomial(10,9)*(5/6)
p8 = binomial(6,1)*(1/6)^8*binomial(10,8)*(5/6)^2
p7 = binomial(6,1)*(1/6)^7*binomial(10,7)*(5/6)^3
p6 = binomial(6,1)*(1/6)^6*binomial(10,6)*(5/6)^4
[(p,numerical_approx(p)) for p in [p10,p9,p8,p7,p6]]

[(1/10077696, 9.92290301275212e-8),
 (25/5038848, 4.96145150637606e-6),
 (125/1119744, 0.000111632658893461),
 (625/419904, 0.00148843545191282),
 (21875/1679616, 0.0130238102042372)]

The probability of rolling 5 of one number and 5 of another number is

In [6]:
p5_2 = binomial(6,2)*(1/6)^10*binomial(10,5)
(p5_2,numerical_approx(p5_2))

(35/559872, 0.0000625142889803384)

The probability of rolling 5 of one number and less than 5 of every other number is

In [7]:
p5_1 = binomial(6,1)*(1/6)^5*binomial(10,5)*((5/6)^5 - (1/6)^5*binomial(5,1))
(p5_1, numerical_approx(p5_1))

(455/5832, 0.0780178326474623)

In [8]:
p5 = p5_1+p5_2
(p5, numerical_approx(p5))

(43715/559872, 0.0780803469364426)

The probability of rolling 4 of one number and 4 of another number is

In [9]:
p4_2 = binomial(6,2)*(1/6)^8*binomial(8,4)*(4/6)^2*binomial(10,2)
(p4_2, numerical_approx(p4_2))

(875/69984, 0.0125028577960677)

The probability of rolling 4 of one number and less than 4 of any other number is

In [10]:
p4_1 = binomial(6,1)*(1/6)^4*binomial(10,4)*((5/6)^6
            - (1/6)^6*binomial(5,1)
            - (1/6)^5*binomial(5,1)*(4/6)*binomial(6,1)
            - (1/6)^4*binomial(5,1)*(4/6)^2*binomial(6,2))
(p4_1, numerical_approx(p4_1))

(125125/419904, 0.297984777472946)

In [11]:
p4 = p4_2 + p4_1
(p4, numerical_approx(p4))

(130375/419904, 0.310487635269014)

The probability of rolling 3 of three numbers and 1 of a fourth number is

In [12]:
p3_3_1 = binomial(6,3)*(1/6)^9*binomial(6,3)*binomial(9,3)*(3/6)*binomial(10,1)
(p3_3_1,numerical_approx(p3_3_1))

(875/52488, 0.0166704770614236)

The probability of rolling 3 of two numbers and less than 3 of any other number is

In [13]:
p3_2 = binomial(6,2)*(1/6)^6*binomial(6,3)*binomial(10,6)*((4/6)^4
            - (1/6)^4*binomial(4,1)
            - (1/6)^3*binomial(4,1)*(3/6)*binomial(4,1))
(p3_2,numerical_approx(p3_2))

(14875/69984, 0.212548582533150)

The probability of rolling 3 of one number and less than 3 of any other number is

In [14]:
p3_1 = binomial(6,1)*(1/6)^3*binomial(10,3)*((5/6)^7
            - (1/6)^7*binomial(5,1)
            - (1/6)^6*binomial(5,1)*(4/6)*binomial(7,1)
            - (1/6)^5*binomial(5,1)*(4/6)^2*binomial(7,2)
            - (1/6)^4*binomial(5,1)*(4/6)^3*binomial(7,3)
            - (1/6)^6*binomial(5,2)*binomial(6,3)*(3/6)*binomial(7,1)
            - (1/6)^3*binomial(5,1)*((4/6)^4
                        - (1/6)^4*binomial(4,1)
                        - (1/6)^3*binomial(4,1)*(3/6)*binomial(4,1))*binomial(7,4))
(p3_1,numerical_approx(p3_1))

(875/2916, 0.300068587105624)

In [15]:
p3 = p3_3_1 + p3_2 + p3_1
(p3,numerical_approx(p3))

(111125/209952, 0.529287646700198)

The probability of rolling 2 of five numbers is

In [16]:
p2_5 = binomial(6,5)*(1/6)^10*binomial(4,2)*binomial(6,2)*binomial(8,2)*binomial(10,2)
(p2_5,numerical_approx(p2_5))

(175/15552, 0.0112525720164609)

The probability of rolling 2 of four numbers and 1 of each of the other two numbers is

In [17]:
p2_4_2 = binomial(6,4)*(1/6)^8*binomial(4,2)*binomial(6,2)*binomial(8,2)*(2/6)*(1/6)*binomial(10,2)
(p2_4_2,numerical_approx(p2_4_2))

(875/15552, 0.0562628600823045)

In [18]:
p2 = p2_5 + p2_4_2
(p2,numerical_approx(p2))

(175/2592, 0.0675154320987654)

In [19]:
p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9 + p10

1

The average number of dice set aside from the first roll of 10 dice is

In [20]:
x = 2*p2 + 3*p3 + 4*p4 + 5*p5 + 6*p6 + 7*p7 + 8*p8 + 9*p9 + 10*p10
(x,numerical_approx(x))

(17357555/5038848, 3.44474669607021)

which agrees with the results of the simulations.

#### Calculating $R$

In [21]:
from functools import cache

@cache
def R(n):
    if n == 0:
        return 0
    denominator = 1 - (5/6)^n*binomial(n,0)
    numerator = 1 + sum([R(n-k)*((1/6)^k*(5/6)^(n-k)*binomial(n,k)) for k in [1..n]])
    return numerator/denominator

In [22]:
[R(k) for k in [1..10]]

[6,
 96/11,
 10566/1001,
 728256/61061,
 3698650986/283994711,
 9438928992/677218157,
 3736509841479198/253815850752893,
 7512121341839614848/487580249296307453,
 443684776375651380574086/27701960514724125452641,
 5002237679837182013478909216/301979071571007691559239541]

In [23]:
k = var('k')
R2(n) = sum(binomial(n,k)*(-1)^(k+1)/(1-(5/6)^k),k,1,n)

In [24]:
[(R(k) - R2(k).simplify(),R2(k).simplify()) for k in [1..10]]

[(0, 6),
 (0, 96/11),
 (0, 10566/1001),
 (0, 728256/61061),
 (0, 3698650986/283994711),
 (0, 9438928992/677218157),
 (0, 3736509841479198/253815850752893),
 (0, 7512121341839614848/487580249296307453),
 (0, 443684776375651380574086/27701960514724125452641),
 (0, 5002237679837182013478909216/301979071571007691559239541)]

And the average number of dice set aside per roll for $N=10$ is

In [25]:
x = 10/((1+R(8))*p2 + (1+R(7))*p3 + (1+R(6))*p4 + (1+R(5))*p5 + (1+R(4))*p6
        + (1+R(3))*p7 + (1+R(2))*p8 + (1+R(1))*p9 + (1+R(0))*p10)
(x,numerical_approx(x))

(447861962188630247878620/687400143797944474874777, 0.651530213121799)

which agrees with the results of the simulations.

### General case

The probability of setting aside $n$ dice from the first roll, when $n > N/2$ is
$\frac{5^{N-n}}{6^{N-1}}\binom{N}{n}$.

When $n < N/6$, the probability is $0$.